# 자전거 대여 데이터

- 2011년부터 2012년까지 2년간의 자전거 대여 데이터
- 캐피털 바이크셰어 회사가 공개한 운행 기록에 다양한 외부 소스에서 얻은 당시 날씨 정보를 조합
- 한 시간 간격으로 기록됨
- 훈련 데이터 : 매달 1일부터 19일 까지의 기록
- 테스트 데이터 : 매달 20일부터 월말까지의 기록
- 피처
    - datetime : 기록 일시(1시간 간격)
    - season : 계절
        - 1 : 봄(1분기)
        - 2 : 여름(2분기)
        - 3 : 가을(3분기)
        - 4 : 겨울(4분기)
        - 공식 문서에는 계절로 설명하고 있지만 실제로는 분기로 나누어져 있음
    - holiday : 공휴일 여부(0 : 공휴일 아님, 1: 공휴일)
    - workingday : 근무일 여부(0 : 근무일 아님, 1 : 근무일)
        - 주말과 공휴일이 아니면 근무일이라고 간주
    - weather : 날씨
        - 1 : 맑음
        - 2 : 옅은 안개, 약간 흐림
        - 3 : 약간의 눈, 약간의 비와 천둥 번개, 흐림
        - 4 : 폭우와 천둥 번개, 눈과 짙은 안개
        - 숫자가 클수록 날씨가 안 좋음
    - temp : 실제 온도
    - atemp : 체감온도
    - humidity : 상대 습도
    - windspeed : 풍속
    - casual : 등록되지 않은 사용자(비회원) 수
    - registered : 등록된 사용자(회원) 수
    - count : 자전거 대여 수량
- 종속변수 : count
- 평가지표 : RMSLE(Root Mean Squared Logarithmic Error), 0.37

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_log_error
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression

In [2]:
def rmsle(y_true, y_pred, convertExp = True):
    '''
    실제 타깃값과 예측값을 인수로 전달하면 RMSLE 수치를 반환하는 함수
    convertExp : 입력 데이터를 지수변환할지 정하는 파라미터
    타깃값으로 log(count)를 사용한 경우에는 지수변환을 해줘야 함
    '''
    # 지수변환
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)

    # 로그변환 후 결측값을 0으로 변환
    log_true = np.nan_to_num(np.log(y_true + 1))
    log_pred = np.nan_to_num(np.log(y_pred + 1))

    # RMSLE 계산
    output = np.sqrt(np.mean((log_true - log_pred)**2))

    return output

In [3]:
df = pd.read_csv("./data/bike/train.csv")
test_df = pd.read_csv("./data/bike/test.csv")

In [4]:
# season을 제거하고 월을 쓰기위한 데이터 변환
df["year"] = df['datetime'].apply(lambda x: int(x[:4]))
df["month"] = df['datetime'].apply(lambda x: int(x[5:7]))
df["time"] = df['datetime'].apply(lambda x: int(x[11:13]))

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
 12  year        10886 non-null  int64  
 13  month       10886 non-null  int64  
 14  time        10886 non-null  int64  
dtypes: float64(3), int64(11), object(1)
memory usage: 1.2+ MB


In [6]:
df[df["count"] < 5]

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,time
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1,2011,1,4
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0,1,1,2011,1,5
6,2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,2,0,2,2011,1,6
7,2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,1,2,3,2011,1,7
28,2011-01-02 04:00:00,1,0,0,2,18.86,22.725,94,12.9980,2,1,3,2011,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10698,2012-12-12 04:00:00,4,0,1,1,10.66,15.150,60,0.0000,0,4,4,2012,12,4
10721,2012-12-13 03:00:00,4,0,1,2,10.66,12.120,56,19.0012,0,2,2,2012,12,3
10771,2012-12-15 05:00:00,4,0,0,2,10.66,15.150,75,0.0000,0,3,3,2012,12,5
10817,2012-12-17 03:00:00,4,0,1,2,14.76,18.940,93,0.0000,0,3,3,2012,12,3


In [7]:
use_df = df.drop("season", axis = 1)
onehot_month = pd.get_dummies(df["month"], dtype = int)
onehot_month.columns = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
onehot_time = pd.get_dummies(df["time"], dtype = int)
onehot_time.columns = ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10",
                        "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23"]

use_df["date"] = df["datetime"].map(lambda x: x.split()[0])
use_df["weekday"] = use_df["date"].map(lambda x: pd.to_datetime(x).weekday())


onehot_season = pd.get_dummies(df["season"], dtype = int)
onehot_season.columns = ["spring", "summer", "fall", "winter"]
onehot_weather = pd.get_dummies(df["weather"], dtype = int)
onehot_weather.columns = ["M", "H", "S", "R"]

use_df = pd.concat([use_df, onehot_month, onehot_time, onehot_season], axis = 1).drop(["month", "time"], axis = 1)
use_df["atemp"] = use_df["atemp"] - use_df["temp"]

In [8]:
use_df.loc[:, "windspeed"] = np.round(df["windspeed"], 2)

In [30]:
x = use_df.drop(["datetime", "casual", "registered", "count", "windspeed","date"], axis = 1)
y1 = use_df["casual"]
y2 = use_df["registered"]
y3 = use_df["count"]

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x.drop("atemp",axis= 1), y3, test_size = 0.3, random_state = 0)

In [32]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor

In [33]:
rf = RandomForestRegressor(max_depth = 25, min_samples_split = 7)
et = ExtraTreesRegressor()
gb = GradientBoostingRegressor()

In [34]:
rf.fit(x_train, np.log1p(y_train))
pred = rf.predict(x_test)
rmsle(y_test, np.expm1(pred), False)

0.3405474650246209

In [35]:
et.fit(x_train, np.log1p(y_train))
pred = et.predict(x_test)
rmsle(y_test, np.expm1(pred), False)

0.35136623520177923

In [36]:
gb.fit(x_train, y_train)
pred = gb.predict(x_test)
rmsle(y_test, pred, False)

C:\Users\sungj\AppData\Local\Temp\ipykernel_18368\2771910247.py:14: RuntimeWarning: invalid value encountered in log
  log_pred = np.nan_to_num(np.log(y_pred + 1))


0.8221463429276744

In [77]:
# season을 제거하고 월을 쓰기위한 데이터 변환
test_df["year"] = test_df['datetime'].apply(lambda x: int(x[:4]))
test_df["month"] = test_df['datetime'].apply(lambda x: int(x[5:7]))
test_df["time"] = test_df['datetime'].apply(lambda x: int(x[11:13]))

In [78]:
use_test_df = test_df.drop("season", axis = 1)
#use_df = use_df.drop("holiday", axis = 1)
onehot_month = pd.get_dummies(test_df["month"], dtype = int)
onehot_month.columns = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
onehot_time = pd.get_dummies(test_df["time"], dtype = int)
onehot_time.columns = ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09", "10",
                        "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23"]
onehot_season = pd.get_dummies(test_df["season"], dtype = int)
onehot_season.columns = ["spring", "summer", "fall", "winter"]


use_test_df = pd.concat([use_test_df, onehot_month, onehot_time, onehot_season], axis = 1).drop(["month", "time"], axis = 1)
#use_test_df["atemp"] = use_test_df["atemp"] - use_test_df["temp"]

In [79]:
use_test_df.loc[:, "windspeed"] = np.round(test_df["windspeed"], 2)

In [80]:
x_train.columns

Index(['holiday', 'workingday', 'weather', 'temp', 'windspeed', 'year', 'Jan',
       'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov',
       'Dec', '00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10',
       '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', 'spring', 'summer', 'fall', 'winter'],
      dtype='object')

In [81]:
use_test_df.drop("datetime", axis = 1).columns

Index(['holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity',
       'windspeed', 'year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',
       'Aug', 'Sep', 'Oct', 'Nov', 'Dec', '00', '01', '02', '03', '04', '05',
       '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '21', '22', '23', 'spring', 'summer', 'fall',
       'winter'],
      dtype='object')

In [82]:
use_test_df = use_test_df.drop(["datetime", "humidity", "atemp"], axis = 1)

In [84]:
pred_test = rf.predict(use_test_df)
pred_test = pd.DataFrame(np.expm1(pred_test), columns = ["count"])

In [85]:
pred_df = pd.concat([test_df["datetime"], pred_test], axis = 1)

In [88]:
pred_df

,datetime,count
0,2011-01-20 00:00:00,8.232073
1,2011-01-20 01:00:00,4.137328
2,2011-01-20 02:00:00,2.153106
3,2011-01-20 03:00:00,3.627462
4,2011-01-20 04:00:00,1.448541
...,...,...
6488,2012-12-31 19:00:00,202.092752
6489,2012-12-31 20:00:00,151.747304
6490,2012-12-31 21:00:00,119.082105
6491,2012-12-31 22:00:00,92.118502


In [91]:
pred_df.to_csv('sample.csv', index=False)

In [23]:
params = {'max_depth': [30], 'min_impurity_decrease': [0.0051], 'min_samples_split': [8]}

In [24]:
gs = GridSearchCV(ExtraTreesRegressor(random_state = 0), params, n_jobs = -1)

In [137]:
gs.fit(x_train, y_train)

GridSearchCV(estimator=ExtraTreesRegressor(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [30], 'min_impurity_decrease': [0.0051],
                         'min_samples_split': [8]})

In [138]:
pred = gs.predict(x_test)
rmsle(y_test, pred, False)

0.3809298251459086

In [89]:
gs.best_params_

{'max_depth': 30, 'min_impurity_decrease': 0.0051, 'min_samples_split': 8}

In [261]:
params = {"min_impurity_decrease" : np.arange(0.00001, 0.01, 0.0001),
         "max_depth" : [25],
         "min_samples_split" : [7]}
gs2 = GridSearchCV(RandomForestRegressor(random_state = 0), params, n_jobs = -1)

In [262]:
gs2.fit(x_train, np.log1p(y_train))

GridSearchCV(estimator=RandomForestRegressor(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [25],
                         'min_impurity_decrease': array([1.00e-05, 1.10e-04, 2.10e-04, 3.10e-04, 4.10e-04, 5.10e-04,
       6.10e-04, 7.10e-04, 8.10e-04, 9.10e-04, 1.01e-03, 1.11e-03,
       1.21e-03, 1.31e-03, 1.41e-03, 1.51e-03, 1.61e-03, 1.71e-03,
       1.81e-03, 1.91e-03, 2.01e-03, 2.11e-03, 2.21e-03, 2.31e-03,
       2.4...
       6.01e-03, 6.11e-03, 6.21e-03, 6.31e-03, 6.41e-03, 6.51e-03,
       6.61e-03, 6.71e-03, 6.81e-03, 6.91e-03, 7.01e-03, 7.11e-03,
       7.21e-03, 7.31e-03, 7.41e-03, 7.51e-03, 7.61e-03, 7.71e-03,
       7.81e-03, 7.91e-03, 8.01e-03, 8.11e-03, 8.21e-03, 8.31e-03,
       8.41e-03, 8.51e-03, 8.61e-03, 8.71e-03, 8.81e-03, 8.91e-03,
       9.01e-03, 9.11e-03, 9.21e-03, 9.31e-03, 9.41e-03, 9.51e-03,
       9.61e-03, 9.71e-03, 9.81e-03, 9.91e-03]),
                         'min_samples_split': [7]})

In [263]:
pred = gs2.predict(x_test)
rmsle(y_test, np.expm1(pred), False)

0.3652472056640648

In [237]:
gs2.best_params_

{'max_depth': 25, 'min_impurity_decrease': 0.001, 'min_samples_split': 4}